In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json

spark = SparkSession.builder.appName("SparkStreamingDemo").getOrCreate()

# subscribe au topic 'persons'
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', 'kafka:9092') \
    .option('subscribe', 'topic-persons') \
    .option('startingOffsets', 'earliest') \
    .load()

query = df_stream.writeStream.format("console").start()
query.awaitTermination(10)

25/10/07 11:53:09 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5f246502-0d98-4175-b6d1-e93425b12fb5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/07 11:53:09 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+--------------------+-------------+---------+------+--------------------+-------------+
| key|               value|        topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-------------+---------+------+--------------------+-------------+
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     0|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     1|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     2|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     3|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     4|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     5|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     6

False

In [29]:
import time
import json
from kafka import KafkaProducer

kafka_url = 'kafka:9092'
producer = KafkaProducer(
    bootstrap_servers=kafka_url,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

topic2 = 'topic-persons'

# Open and read the JSON file line by line
with open('data/persons.json', 'r', encoding='utf-8') as file:
    data = json.load(file)  # charge le tableau complet

    for i, obj in enumerate(data):
        if i >= 25:
            break
        producer.send(topic2, value=json.dumps(obj))
        print(f'Envoyé: {obj}')
        time.sleep(1)

producer.flush()
producer.close()


Envoyé: {'id': 1, 'name': 'Alice', 'age': 30, 'city': 'Paris'}
-------------------------------------------
Batch: 1
-------------------------------------------
+----+--------------------+-------------+---------+------+--------------------+-------------+
| key|               value|        topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-------------+---------+------+--------------------+-------------+
|null|[22 7B 5C 22 69 6...|topic-persons|        0|    75|2025-10-07 11:53:...|            0|
+----+--------------------+-------------+---------+------+--------------------+-------------+

-------------------------------------------
Batch: 1
-------------------------------------------
-------------------------------------------
Batch: 51
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|"{\"id\": 1, \"na...|
+--------------------+

+----+--------------------+-------------+---------+------+

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("SparkStreamingDemo").getOrCreate()

df_stream = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "topic-persons") \
    .option("startingOffsets", "earliest") \
    .load()

# Convert Kafka value to string
df_stream = df_stream.select(col("value").cast("string"))

# Write to console
query = df_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination(60)


# Faire une execution ligne par ligne

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("SparkStreamingDemo").getOrCreate()

df_stream = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "topic-persons") \
    .option("startingOffsets", "earliest") \
    .load()

# Convert Kafka value to string
lines = df_stream.selectExpr("cast(value AS STRING)")

def foreach_batch (batch_df, batch_id):
    print(f"Batch ID: {batch_id}")
    batch_df.show(truncate = False)

query = lines.writeStream.foreachBatch(foreach_batch).start()
query.awaitTermination(20)

Batch ID: 15
+--------------------------------------------------------------------------+
|value                                                                     |
+--------------------------------------------------------------------------+
|"{\"id\": 15, \"name\": \"Olivier\", \"age\": 39, \"city\": \"Bordeaux\"}"|
+--------------------------------------------------------------------------+

Batch ID: 16
+----------------------------------------------------------------------+
|value                                                                 |
+----------------------------------------------------------------------+
|"{\"id\": 16, \"name\": \"Pauline\", \"age\": 34, \"city\": \"Nice\"}"|
+----------------------------------------------------------------------+

Batch ID: 17
+----------------------------------------------------------------------------+
|value                                                                       |
+--------------------------------------------------

In [2]:
kafka_server = 'kafka:9092'

In [3]:
import json
from kafka.admin import KafkaAdminClient, NewTopic
from kafka import KafkaProducer

topic = 'film'

admin = KafkaAdminClient(bootstrap_servers=kafka_server)
topic_list = admin.list_topics()
if topic not in topic_list:
    new_topic = NewTopic(name=topic, num_partitions=1, replication_factor=1)
    admin.create_topics(new_topics=[new_topic])

producer = KafkaProducer(bootstrap_servers=[kafka_server], value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [ ]:

import json
from kafka.admin import KafkaAdminClient, NewTopic
from kafka import KafkaProducer

topic = 'topic-persons'

admin = KafkaAdminClient(bootstrap_servers=kafka_server)
topic_list = admin.list_topics()
if topic not in topic_list:
    new_topic = NewTopic(name=topic, num_partitions=1, replication_factor=1)
    admin.create_topics(new_topics=[new_topic])

producer = KafkaProducer(bootstrap_servers=[kafka_server], value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkStreamingDemo").getOrCreate()
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', kafka_server) \
    .option('subscribe', 'topic-persons') \
    .option('startingOffsets', 'earliest') \
    .load()

# Afficher uniquement la valeur (value) du message Kafka en string
lines = df_stream.selectExpr("CAST(value AS STRING)")

query = lines.writeStream.format("console").start()
query.awaitTermination(10)

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkExample3").getOrCreate()
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', kafka_server) \
    .option('subscribe', 'topic-persons') \
    .option('startingOffsets', 'earliest') \
    .load()

def process_batch(batch_df, batch_id):
    batch_df.write.format("jdbc") \
        .option("url", "jdbc:postgresql://postgres:5432/events") \
        .option("dbtable", "public.persons") \
        .option("user", "app") \
        .option("password", "1234") \
        .mode("append") \
        .save()

query = df_stream.writeStream \
    .foreachBatch(process_batch) \
    .start()
query.awaitTermination(100)
query.stop()

25/10/07 11:16:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/10/07 11:16:01 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-590218b6-822a-4b1d-9a28-98075cba031a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/07 11:16:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [25]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkExample3").getOrCreate()
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', kafka_server) \
    .option('subscribe', topic) \
    .option('startingOffsets', 'earliest') \
    .load()

def process_batch(batch_df, batch_id):
    batch_df.write.format("jdbc") \
        .option("url", "jdbc:postgresql://postgres:5432/events") \
        .option("dbtable", "public.raw_persons") \
        .option("user", "app") \
        .option("password", "1234") \
        .mode("append") \
        .save()

query = df_stream.writeStream \
    .foreachBatch(process_batch) \
    .start()
query.awaitTermination(10)
query.stop()

25/10/07 11:51:03 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2940aa50-538c-4541-b0db-c7996c427fba. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/07 11:51:03 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/10/07 11:51:03 WARN NetworkClient: [AdminClient clientId=adminclient-21] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:51:03 WARN NetworkClient: [AdminClient clientId=adminclient-21] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:51:04 WARN NetworkClient: [AdminClient clientId=adminclient-21] Connection to node -1 (localhost/12

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("SparkExample3").getOrCreate()
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', kafka_server) \
    .option('subscribe', topic) \
    .option('startingOffsets', 'earliest') \
    .load()

schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("age", IntegerType(), False),
    StructField("city", StringType(), False)
])

lines = df_stream.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.*")

def process_batch(batch_df, batch_id):
    batch_df.write.format("jdbc") \
        .option("url", "jdbc:postgresql://postgres:5432/events") \
        .option("dbtable", "public.persons") \
        .option("user", "app") \
        .option("password", "1234") \
        .mode("append") \
        .save()

query = lines.writeStream \
    .foreachBatch(process_batch) \
    .start()
query.awaitTermination(10)
query.stop()

25/10/07 11:50:41 WARN NetworkClient: [AdminClient clientId=adminclient-20] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:50:41 WARN NetworkClient: [AdminClient clientId=adminclient-19] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:50:42 WARN NetworkClient: [AdminClient clientId=adminclient-20] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:50:42 WARN NetworkClient: [AdminClient clientId=adminclient-19] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:50:43 WARN NetworkClient: [AdminClient clientId=adminclient-20] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:50:43 WARN NetworkClient: [AdminClient clientId=adminclient-19] Connection to node -1 (l

KeyboardInterrupt: 

25/10/07 11:51:03 WARN NetworkClient: [AdminClient clientId=adminclient-20] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
